![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train Domain-specific Multiclass Classifiers

In this notebook, you will learn how to use Spark NLP and Finance NLP to train custom multiclass classification models.

In [ ]:
from johnsnowlabs import nlp, finance, viz

## Introduction

Although John Snow Labs provides mnay pretrained models that cover different applications in the financial domain, there are still problems that are specific to companies or practitioners. For such cases, it is possible to train a new custom model using Finance NLP annotators:

- `ClassifierDLApproach`: Trains a multilabel model (predicts one class out of a predefined set of classes) or binary classification
- `MultiClassifierDLApproach`: Trains a mutilabel model (predicts one or more classes for each document)

The input to are Sentence Embeddings such as the state-of-the-art [UniversalSentenceEncoder](https://nlp.johnsnowlabs.com/docs/en/transformers#universalsentenceencoder), [BertSentenceEmbeddings](https://nlp.johnsnowlabs.com/docs/en/transformers#bertsentenceembeddings) or [SentenceEmbeddings](https://nlp.johnsnowlabs.com/docs/en/annotators#sentenceembeddings).

To train a custom model, you need labeled data with at least the columns

```
| TEXT | LABELS (list) |
```

## Multiclass Classifier Training

The `ClassifierDLApproach` annotator trains a multiclass model, where the predictions is one category out of a predifined set of categories that are present in the training data.

## Loading the data

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/finance-nlp/data/finance_clf_data.csv

dbutils.fs.cp("file:/databricks/driver/finance_clf_data.csv", "dbfs:/") 

Out[2]: True

In [ ]:
import pandas as pd
df = pd.read_csv('finance_clf_data.csv')
print(f"Shape of the full dataset: {df.shape}")

Shape of the full dataset: (6128, 3)

In [ ]:
df.head()

Out[4]:

,text,label,len
0,Presently we do not believe any U S or State r...,business,402
1,\nnetwork outages or performance degradation ...,risk_factors,496
2,Available Information\nOur reports filed with ...,business,356
3,\n 42 530\n \n \n \n \n \n 42 530\nTotal liab...,financial_statements,359
4,8\nTable of Contents\ndevelopment employee eng...,business,582


In [ ]:
df['label'].value_counts()

Out[5]: risk_factors 1926
financial_statements 1888
business 970
financial_conditions 346
form_10k_summary 240
executives_compensation 155
controls_procedures 138
equity 111
market_risk 100
executives 73
legal_proceedings 51
properties 48
security_ownership 46
exhibits 36
Name: label, dtype: int64

Since the deep learning models can take some time to train, we will limit our dataset to a smaller number of observations in order to illustrate how to use Spark NLP and Finance NLP annotators and pipelines to train the model, but without having to wait too much.

Please note that the quality and the quantity of training data is very relevant to the obtained trianed model, and the results we obtain here are for illustration purposes only. To obtain a more realistic model, pelase consider using the full dataset or addin extra observations from different sources.

In [ ]:
from sklearn.model_selection import train_test_split

# The top 3 categories (number of observations) 
filter_classes = [
    "risk_factors",
    "financial_statements",
    "business"
]

# We make a random sample with 500 observations
df = df.loc[df.label.isin(filter_classes)].sample(500)

# Stratify split for train and test datasets
train_data, test_data = train_test_split(
    df, train_size=0.9, stratify=df.label, random_state=42
)

# Send to spark
train = spark.createDataFrame(train_data) 
test = spark.createDataFrame(test_data)

In [ ]:
from pyspark.sql.functions import col

train.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
 label|count|
+--------------------+-----+
financial_statements| 188|
 risk_factors| 181|
 business| 81|
+--------------------+-----+

In [ ]:
from pyspark.sql.functions import col

test.groupBy("label").count().orderBy(col("count").desc()).show()

+--------------------+-----+
 label|count|
+--------------------+-----+
financial_statements| 21|
 risk_factors| 20|
 business| 9|
+--------------------+-----+

## Train with Universal Encoder

In [ ]:
%fs mkdirs file:/dbfs/multiclass_use

res3: Boolean = true

In [ ]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

embeddings = (
    nlp.UniversalSentenceEncoder.pretrained()
    .setInputCols("document")
    .setOutputCol("sentence_embeddings")
)

classifierdl = (
    finance.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(30)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("file:/dbfs/multiclass_use")
    .setLr(0.001)
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, classifierdl])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [ ]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 109 ms, sys: 26.3 ms, total: 135 ms
Wall time: 24.4 s

In [ ]:
import os
log_file_name = os.listdir("/dbfs/multiclass_use")[0]

with open("/dbfs/multiclass_use/"+log_file_name, "r") as log_file:
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.001 - batch_size: 4 - training_examples: 450 - classes: 3
Epoch 0/30 - 0.84s - loss: 108.63804 - acc: 0.60491073 - batches: 113
Epoch 1/30 - 0.44s - loss: 93.2874 - acc: 0.73883927 - batches: 113
Epoch 2/30 - 0.44s - loss: 85.61947 - acc: 0.84375 - batches: 113
Epoch 3/30 - 0.43s - loss: 81.13512 - acc: 0.87053573 - batches: 113
Epoch 4/30 - 0.42s - loss: 78.461784 - acc: 0.88616073 - batches: 113
Epoch 5/30 - 0.41s - loss: 76.84228 - acc: 0.8995536 - batches: 113
Epoch 6/30 - 0.42s - loss: 75.85659 - acc: 0.90401787 - batches: 113
Epoch 7/30 - 0.44s - loss: 75.21753 - acc: 0.91071427 - batches: 113
Epoch 8/30 - 0.46s - loss: 74.76673 - acc: 0.9129464 - batches: 113
Epoch 9/30 - 0.43s - loss: 74.406265 - acc: 0.9151786 - batches: 113
Epoch 10/30 - 0.43s - loss: 74.099884 - acc: 0.91741073 - batches: 113
Epoch 11/30 - 0.42s - loss: 73.82788 - acc: 0.91741073 - batches: 113
Epoch 12/30 - 0.44s - loss: 73.58166 - acc: 0.91964287 - batches: 113
Epoch 13/30 - 0.44s - loss: 73.363335 - acc: 0.91964287 - batches: 113
Epoch 14/30 - 0.46s - loss: 73.169525 - acc: 0.921875 - batches: 113
Epoch 15/30 - 0.44s - loss: 73.00098 - acc: 0.92410713 - batches: 113
Epoch 16/30 - 0.42s - loss: 72.852615 - acc: 0.92410713 - batches: 113
Epoch 17/30 - 0.45s - loss: 72.72597 - acc: 0.92633927 - batches: 113
Epoch 18/30 - 0.43s - loss: 72.61168 - acc: 0.92633927 - batches: 113
Epoch 19/30 - 0.45s - loss: 72.51238 - acc: 0.92633927 - batches: 113
Epoch 20/30 - 0.42s - loss: 72.420494 - acc: 0.92633927 - batches: 113
Epoch 21/30 - 0.42s - loss: 72.33612 - acc: 0.92633927 - batches: 113
Epoch 22/30 - 0.43s - loss: 72.25952 - acc: 0.92633927 - batches: 113
Epoch 23/30 - 0.42s - loss: 72.18505 - acc: 0.9285714 - batches: 113
Epoch 24/30 - 0.40s - loss: 72.11643 - acc: 0.9308036 - batches: 113
Epoch 25/30 - 0.40s - loss: 72.04982 - acc: 0.9308036 - batches: 113
Epoch 26/30 - 0.39s - loss: 71.98364 - acc: 0.9308036 - batches: 113
Epoch 27/30 - 0.41s - loss: 71.91841 - acc: 0.93303573 - batches: 113
Epoch 28/30 - 0.39s - loss: 71.85373 - acc: 0.93526787 - batches: 113
Epoch 29/30 - 0.42s - loss: 71.78748 - acc: 0.93526787 - batches: 113

In [ ]:
preds = clf_pipelineModel.transform(test)

In [ ]:
preds_df = preds.select("label", "text", "class.result").toPandas()
preds_df.head()

Out[43]:

,label,text,result
0,financial_statements,740 In addition the Company not record a cumul...,[financial_statements]
1,risk_factors,Failure to effectively develop and expand our ...,[risk_factors]
2,risk_factors,valuable management resources improving our op...,[risk_factors]
3,business,Overall Trends in the TMT Market\n \nConvergen...,[business]
4,financial_statements,As of January 31 2021 no amounts were outstand...,[financial_statements]


In [ ]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

precision recall f1-score support

 business 0.71 0.56 0.63 9
financial_statements 0.91 0.95 0.93 21
 risk_factors 0.86 0.90 0.88 20

 accuracy 0.86 50
 macro avg 0.83 0.80 0.81 50
 weighted avg 0.85 0.86 0.85 50

### Saving & loading back the trained model

In [ ]:
clf_pipelineModel.stages

Out[46]: [DocumentAssembler_c0a55a1ffc4f,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 FinanceClassifierDLModel_62ff309042e1]

In [ ]:
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/Clf_Use')

In [ ]:
# Load back  saved Classifier Model
ClfModel = finance.ClassifierDLModel.load('dbfs:/databricks/driver/models/Clf_Use')

In [ ]:
ld_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings, ClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [ ]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [ ]:
ld_preds_df = ld_preds.select("text", "label", "class.result").toPandas()

In [ ]:
ld_preds_df.head()

Out[52]:

,text,label,result
0,740 In addition the Company not record a cumul...,financial_statements,[financial_statements]
1,Failure to effectively develop and expand our ...,risk_factors,[risk_factors]
2,valuable management resources improving our op...,risk_factors,[risk_factors]
3,Overall Trends in the TMT Market\n \nConvergen...,business,[business]
4,As of January 31 2021 no amounts were outstand...,financial_statements,[financial_statements]


## Train with Bert Embeddings

We do not have Financial Sentence Embeddings yet, But we can use the Financial Word Embeddings and then average them. Since this model takes a long time to train, we will train for only one epoch.

In [ ]:
%fs 
mkdirs file:/dbfs/multiclass_bert

res8: Boolean = true

In [ ]:
embeddings = (
    nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base", "en")
    .setInputCols(["document", "token"])
    .setOutputCol("embeddings")
)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

In [ ]:
document_assembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")
)

tokenizer = nlp.Tokenizer().setInputCols(["document"]).setOutputCol("token")

embeddingsSentence = (
    nlp.SentenceEmbeddings()
    .setInputCols(["document", "embeddings"])
    .setOutputCol("sentence_embeddings")
    .setPoolingStrategy("AVERAGE")
)

classifierdl = (
    finance.ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("class")
    .setLabelColumn("label")
    .setMaxEpochs(1)
    .setLr(0.001)
    .setEnableOutputLogs(True)
    .setOutputLogsPath("dbfs:/multiclass_bert")
    .setBatchSize(4)
    .setDropout(0.15)
)

clf_pipeline = nlp.Pipeline(
    stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, classifierdl]
)

In [ ]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 180 ms, sys: 90.2 ms, total: 270 ms
Wall time: 47.2 s

In [ ]:
log_files = os.listdir("/dbfs/multiclass_bert")
with open("/dbfs/multiclass_bert/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 1 - learning_rate: 0.001 - batch_size: 4 - training_examples: 450 - classes: 3
Epoch 0/1 - 0.79s - loss: 98.23462 - acc: 0.68973213 - batches: 113

In [ ]:
preds = clf_pipelineModel.transform(test)

In [ ]:
preds_df = preds.select("label", "text", "class.result").toPandas()

In [ ]:
preds_df.head()

Out[118]:

,label,text,result
0,financial_statements,740 In addition the Company not record a cumul...,[financial_statements]
1,risk_factors,Failure to effectively develop and expand our ...,[risk_factors]
2,risk_factors,valuable management resources improving our op...,[risk_factors]
3,business,Overall Trends in the TMT Market\n \nConvergen...,[financial_statements]
4,financial_statements,As of January 31 2021 no amounts were outstand...,[financial_statements]


In [ ]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


precision recall f1-score support

 business 0.00 0.00 0.00 9
financial_statements 0.77 0.95 0.85 21
 risk_factors 0.83 1.00 0.91 20

 accuracy 0.80 50
 macro avg 0.53 0.65 0.59 50
 weighted avg 0.66 0.80 0.72 50

### Save model

In [ ]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/MultiClfBert')
